In [1]:
import re
import pandas as pd
import random
import numpy as np

filename = "C:\\Users\\mustafa\\Documents\\test.txt"
filename2 = "C:\\Users\\mustafa\\Documents\\test2.txt"

filenames = [filename, filename2]

In [2]:
def extract_all_shingles(filenames):
    
    k4_dict = {}
    k5_dict = {}
    k8_dict = {}
    
    for filename in filenames:
        
        file = open(filename, "r") 
        text = file.read()
        text = re.sub('[^0-9a-zA-Z]+', ' ', text)
        text = re.sub('  ', ' ', text)
        text = text.lower()

        for i in range(len(text)):
            k4 = text[i : i+4]
            k5 = text[i : i+5]
            k8 = text[i : i+8]

            if k4 not in k4_dict:
                k4_dict[k4] = 0
            if k5 not in k5_dict:
                k5_dict[k5] = 0
            if k8 not in k8_dict:
                k8_dict[k8] = 0

    return k4_dict, k5_dict, k8_dict   

    
def shingle(filename, k4_dict, k5_dict, k8_dict):
      
    k4_dict1 = k4_dict.copy()
    k5_dict1 = k5_dict.copy()
    k8_dict1 = k8_dict.copy()
                
    file = open(filename, "r") 
    text = file.read()
    text = re.sub('[^0-9a-zA-Z]+', ' ', text)
    text = re.sub('  ', ' ', text)
    text = text.lower()

    for i in range(len(text)):
        k4 = text[i : i+4]
        k5 = text[i : i+5]
        k8 = text[i : i+8]

        if k4 in k4_dict1:
            k4_dict1[k4] = 1
        if k5 in k5_dict1:
            k5_dict1[k5] = 1
        if k8 in k8_dict1:
            k8_dict1[k8] = 1

    return k4_dict1, k5_dict1, k8_dict1     


def jaccard2(x,y):
    counter = 0
    for key in x:
        if x[key] == 1 and y[key] == 1:
            counter = counter + 1      
    return (counter / len(x))     


def jaccard(x, y):
    T1 = x == 1
    T2 = y == 1

    F1 = x == 0
    F2 = y == 0

    total = len(x)
    ratio = np.sum(T1 == T2) / (total - np.sum(F1 == F2))
    return ratio
            
    
def hash(a):
    def sub_hash(x, m = 991):
        return (a*x + 1) % m 
    return sub_hash

In [3]:
k4_dict, k5_dict, k8_dict = extract_all_shingles(filenames)

k4_matrix = np.zeros((len(k4_dict), len(filenames)))
k5_matrix = np.zeros((len(k5_dict), len(filenames)))
k8_matrix = np.zeros((len(k8_dict), len(filenames)))

for i, filename in enumerate(filenames):
    k4_dict1, k5_dict1, k8_dict1 = shingle(filename, k4_dict, k5_dict, k8_dict)
    k4_matrix[:,i] = np.fromiter(k4_dict1.values(), dtype=float)
    k5_matrix[:,i] = np.fromiter(k5_dict1.values(), dtype=float)
    k8_matrix[:,i] = np.fromiter(k8_dict1.values(), dtype=float)

In [4]:
jaccard(k4_matrix[:, 0], k4_matrix[:, 1])

0.17777777777777778

In [5]:
hashx = []
rand_hash = random.sample(range(1, 991), 100)

for i in range(100):
    hashx.append(hash(rand_hash[i]))

In [6]:
'''
def h1(x):
    return x % 5 + 1

def h2(x):
    return (2*x + 1) % 5 + 1

hashx = []
hashx.append(h1)
hashx.append(h2)
'''

'\ndef h1(x):\n    return x % 5 + 1\n\ndef h2(x):\n    return (2*x + 1) % 5 + 1\n\nhashx = []\nhashx.append(h1)\nhashx.append(h2)\n'

In [7]:
'''
matris = np.zeros((5, 2))
matris_hash = np.zeros((5, 2))
signature_matrix = np.full((2, 2), 999)

matris[0, 0] = 1
matris[1, 1] = 1
matris[2, 0] = 1
matris[2, 1] = 1
matris[3, 0] = 1
matris[4, 1] = 1
'''

'\nmatris = np.zeros((5, 2))\nmatris_hash = np.zeros((5, 2))\nsignature_matrix = np.full((2, 2), 999)\n\nmatris[0, 0] = 1\nmatris[1, 1] = 1\nmatris[2, 0] = 1\nmatris[2, 1] = 1\nmatris[3, 0] = 1\nmatris[4, 1] = 1\n'

In [8]:
shingle_size = len(k4_dict)

nrow = shingle_size
ncol = len(filenames)
nhash = len(hashx)

# matris shingle var yok matrisi
matris = k4_matrix
matris_hash = np.zeros((nrow, nhash))
signature_matrix = np.full((nhash, ncol), 9999999)

In [9]:
for i in range(matris_hash.shape[0]):
    for j in range(matris_hash.shape[1]):
        h = hashx[j]
        matris_hash[i, j] = h(i + 1)
    
    for j in range(matris.shape[1]):
        if(matris[i, j] == 1):
            for k in range(matris_hash.shape[1]):
                if(signature_matrix[k, j] > matris_hash[i, k]):
                    signature_matrix[k, j] = matris_hash[i, k]


print("matris hash")
print(matris_hash)
print("matris")
print(matris)
print("signature matrix")
print(signature_matrix)

matris hash
[[ 816.  506.  931. ...,  865.  495.  924.]
 [ 640.   20.  870. ...,  738.  989.  856.]
 [ 464.  525.  809. ...,  611.  492.  788.]
 ..., 
 [ 699.   14.   74. ...,   88.  677.  586.]
 [ 523.  519.   13. ...,  952.  180.  518.]
 [ 347.   33.  943. ...,  825.  674.  450.]]
matris
[[ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.

In [10]:
signature_matrix

array([[  6,   6],
       [  6,   6],
       [  0,  13],
       [ 20,  18],
       [ 24,  21],
       [  5,  37],
       [ 11,  33],
       [ 12,  35],
       [  0,  39],
       [  4,   4],
       [  3,  18],
       [ 15,   0],
       [  0,  43],
       [  6,   2],
       [  5,  30],
       [  0,  11],
       [  2,   9],
       [  4,  12],
       [  8,  17],
       [  8,   6],
       [  0,  34],
       [  5,  17],
       [  2,   2],
       [  8,   5],
       [  4,  10],
       [  0,  75],
       [ 10,   6],
       [  0,  29],
       [  2,   7],
       [  0,  24],
       [ 10,   0],
       [  0,   0],
       [  6,  39],
       [  8,   4],
       [  3,  10],
       [  2,   7],
       [  2,  24],
       [ 16,  24],
       [  3,  27],
       [  3,  24],
       [  4,  20],
       [  4,  28],
       [  0,  15],
       [  3,   7],
       [  0,  14],
       [  4,  22],
       [  5,  12],
       [  0,  10],
       [  2,  31],
       [  0,  18],
       [  8,  53],
       [  8,  35],
       [  0,